In [3]:
import sys
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install torch


     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     --------------------------------------- 0.8/172.3 MB 24.1 MB/s eta 0:00:08
     --------------------------------------- 1.6/172.3 MB 20.5 MB/s eta 0:00:09
      -------------------------------------- 2.7/172.3 MB 21.9 MB/s eta 0:00:08
      -------------------------------------- 4.4/172.3 MB 25.5 MB/s eta 0:00:07
     - ------------------------------------- 6.7/172.3 MB 30.6 MB/s eta 0:00:06
     -- ------------------------------------ 9.6/172.3 MB 36.2 MB/s eta 0:00:05
     --- ---------------------------------- 13.7/172.3 MB 65.6 MB/s eta 0:00:03
     --- ---------------------------------- 17.7/172.3 MB 81.8 MB/s eta 0:00:02
     ---- --------------------------------- 20.7/172.3 MB 81.8 MB/s eta 0:00:02
     ----- -------------------------------- 24.4/172.3 MB 81.8 MB/s eta 0:00:02
     ------ ------------------------------- 28.1/172.3 MB 81.8 MB/s eta 0:00:02
     ------ ------------------------------- 31.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from sklearn.model_selection import train_test_split

In [5]:
dataset = pd.read_csv('Data/all_data.csv')

#subtract the columns L_0_x, L_0_y, L_0_z from all the columns
for i in range(0, 21):
    dataset[f'L_{i}_x'] = dataset[f'L_{i}_x'] - dataset['L_0_x']
    dataset[f'L_{i}_y'] = dataset[f'L_{i}_y'] - dataset['L_0_y']
    # dataset[f'L_{i}_z'] = dataset[f'L_{i}_z'] - dataset['L_0_z']
    dataset[f'R_{i}_x'] = dataset[f'R_{i}_x'] - dataset['R_0_x']
    dataset[f'R_{i}_y'] = dataset[f'R_{i}_y'] - dataset['R_0_y']
    # dataset[f'R_{i}_z'] = dataset[f'R_{i}_z'] - dataset['R_0_z']

#write datatset to csv
dataset.to_csv('Data/relative_data.csv', index=False)

X = dataset.iloc[:, 1:]
Y = dataset.iloc[:, 0]

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# # Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

print(X_train)
print(Y_train)


      L_0_x  L_0_y         L_0_z     L_1_x     L_1_y     L_1_z     L_2_x  \
2798    0.0    0.0  5.491473e-08  0.664706  0.476913  0.030838  0.659060   
281     0.0    0.0  5.343833e-08  0.242040  0.385963 -0.016513  0.267439   
437     0.0    0.0 -2.759861e-07  0.116181  0.583054 -0.027148  0.113744   
1963    0.0    0.0  2.474445e-07  0.340919  0.749715 -0.019666  0.377801   
2817    0.0    0.0  3.515295e-07  0.699664  0.501086 -0.010196  0.685714   
...     ...    ...           ...       ...       ...       ...       ...   
40      0.0    0.0 -1.276113e-07  0.179122  0.377849 -0.028756  0.194402   
1256    0.0    0.0 -3.123723e-09  0.249787  0.576319 -0.023095  0.274000   
1797    0.0    0.0  5.037747e-07  0.322179  0.700961 -0.021987  0.364543   
2192    0.0    0.0  4.708281e-07  0.371532  0.760761 -0.038503  0.422135   
73      0.0    0.0 -6.202112e-08  0.143765  0.625317 -0.023162  0.141084   

         L_2_y     L_2_z     L_3_x  ...    R_17_z    R_18_x    R_18_y  \
2798  0.417632

In [14]:
X_train_tensor = torch.tensor(X_train.values).float()
X_test_tensor = torch.tensor(X_test.values).float()
Y_train_tensor = torch.tensor(Y_train.values).long()
Y_test_tensor = torch.tensor(Y_test.values).long()

In [15]:
print(X_train_tensor.shape)
print(Y_train_tensor.shape)


torch.Size([2364, 126])
torch.Size([2364])


In [16]:
batch_size = 64
n_iters = 10000
num_epochs = n_iters / (len(X_train) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(X_train_tensor,Y_train_tensor)
test = torch.utils.data.TensorDataset(X_test_tensor,Y_test_tensor)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

(tensor([[ 0.0000e+00,  0.0000e+00,  5.4915e-08,  ...,  4.1597e-01,
           4.4030e-01,  1.6427e-02],
         [ 0.0000e+00,  0.0000e+00,  5.3438e-08,  ...,  6.9344e-01,
           7.9241e-01, -1.6040e-02],
         [ 0.0000e+00,  0.0000e+00, -2.7599e-07,  ...,  7.9771e-01,
           7.6943e-01, -8.1739e-02],
         ...,
         [ 0.0000e+00,  0.0000e+00,  5.0377e-07,  ...,  8.1093e-01,
           6.0954e-01, -6.9597e-02],
         [ 0.0000e+00,  0.0000e+00,  4.7083e-07,  ...,  7.9166e-01,
           3.8474e-01, -6.8406e-02],
         [ 0.0000e+00,  0.0000e+00, -6.2021e-08,  ...,  8.4848e-01,
           6.3071e-01, -6.5983e-02]]),
 tensor([2, 0, 0,  ..., 1, 1, 0]))

In [ ]:
class GestureFNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GestureFNN, self).__init__()
        # Linear function 1: 126 --> 84
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)

        # Non-linearity 1
        self.sigmoid = torch.nn.Sigmoid()

        # Linear function 2: 84 --> 3
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)

        # Non-linearity 1
        out = self.sigmoid(out)

        # Linear function 2 (readout)
        out = self.fc2(out)
        return out

